<a href="https://colab.research.google.com/github/ibonetc/AI_Deep-Learning/blob/main/Transformers_HuggingFace_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers en texto

- Generación de texto
- Resúmenes de texto
- Traducción de texto
- Análisis de sentimientos
- Traducir audios

**Nota**: Intalar estas bibliotecas y reiniciar el entorno

In [1]:
!pip install transformers[sentencepiece]

In [2]:
!pip install transformers
!pip install transformers datasets

In [3]:
!pip install -q -U accelerate peft
!pip install -q bitsandbytes datasets sentencepiece einops gradio trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 43.5 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)

5.0.0


In [5]:
from transformers import pipeline, set_seed

## Generador

### Generador en inglés

In [ ]:
generator = pipeline("text-generation", model="openai-community/gpt2",device="cuda")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
generator("I'm a goddess", max_length=300, num_return_sequences=5,pad_token_id=50256)

[{'generated_text': 'I\'m a goddess, and I\'m your partner."\n\n"Are you trying to talk something up a lot with this man?" The man snorted. "How embarrassing! And do I have to ask, who does any of this involve? You could give her your \'fancy\' name, but you know the real reason, because you know me."\n\nFantasy: What you know is not only bad but downright dangerous. In a world where people are so fascinated with what\'s "good" about living life\'s pleasures that their lives are turned upside-down with every revelation they find, their thoughts are a constant source of stress. People are so focused on making the world more, not less, valuable that they can\'t make a conscious decision about how and when to live their lives. The problem with fantasy is that, as a person, these are deeply harmful and dangerous ideas, which come about quickly because what they really mean is that it\'s very difficult to make up your mind that everything they say is not true on account of what they have in

#### Temperatura

La temperatura en un modelo de lenguaje grande (LLM) es un parámetro que controla el nivel de "aleatoriedad" o "creatividad" de las respuestas generadas por el modelo. Este parámetro ajusta la probabilidad de seleccionar palabras menos comunes en lugar de las palabras más probables, permitiendo generar respuestas más variadas o mantenerlas más enfocadas.

Valores bajos de temperatura (cercanos a 0): Hacen que el modelo sea más determinista, eligiendo con mayor frecuencia las palabras de mayor probabilidad. Esto puede resultar en respuestas más directas y menos creativas.
Valores altos de temperatura (cercanos a 1 o superiores): Aumentan la variabilidad en la elección de palabras, dando lugar a respuestas más creativas y diversas, aunque también pueden ser menos coherentes.

Valor predeterminado 1.

In [ ]:
prompt = "The future of artificial intelligence in healthcare is"

response_low_temp = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.1, pad_token_id=50256)
for r in response_low_temp:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---


In [ ]:
response_low_temp = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.5, pad_token_id=50256)
for r in response_low_temp:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain. In the first half of the 20th century, medical technology was more advanced than ever. Today, the medical profession has become increasingly sophisticated, and the number of doctors in the US has doubled. In
---
The future of artificial intelligence in healthcare is a difficult one, and it's not going to be easy.

"The only way to get there is to move forward on the fundamental question of which technologies are most important and which are the most effective
---
The future of artificial intelligence in healthcare is in the hands of a small group of researchers. The group, led by a team at the University of Michigan, is led by Dr. Charles J. J. Moore, a professor of electrical engineering and computer
---


In [ ]:
response_temp = generator(prompt, max_length=50, num_return_sequences=3, temperature=1.5, pad_token_id=50256)
for r in response_temp:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is in uncertain and potentially dangerous terms because many health authorities cannot provide effective safeguards."—Eileen Murphy Salk, PhD, Chief Scientist


If you or someone in your life you trust who isn't also someone
---
The future of artificial intelligence in healthcare is more clear. Google acquired General Electric, maker of the Autonomously Engineered Mechanical Parts, in May, according to The Next Web. Google already runs research on using artificial intelligence and real brain tissue in medicine
---
The future of artificial intelligence in healthcare is at stake because every single program or service the government provides to AI will be increasingly tied to one specific user base. There are very big implications for social welfare, for public safety, for healthcare delivery. This also
---


####Otros parámetros clave en LLMs
**Top-k Sampling**: Este parámetro establece un límite en el número de palabras entre las que el modelo puede elegir en cada paso de generación. Con top-k = 10, el modelo seleccionará entre las 10 palabras con mayor probabilidad, generando respuestas controladas pero un poco más variadas. Valor predeterminado 50

**Top-p** (Nucleus Sampling): En vez de limitar el número de palabras (como en top-k), se establece una probabilidad acumulada. Por ejemplo, con top-p = 0.9, el modelo elige palabras hasta que la probabilidad acumulada alcanza el 90%. Esto permite flexibilidad sin restringir a un número fijo de opciones. Valor predeterminado 1

**Max Length**: Limita la cantidad máxima de tokens que el modelo puede generar. Es útil para evitar respuestas demasiado largas o cortar respuestas en un número específico de tokens. Generalemente está en un rango de 512 y 1024

**Repetition Penalty**: Penaliza la repetición de palabras en las respuestas, útil para evitar respuestas repetitivas y promover la diversidad en el texto generado. Valor predetermiando 1

**Variando top_k**

**top_k = 1**: El modelo solo considera primera palabra más probables en cada paso. Esto limita la creatividad, haciendo las respuestas más coherentes y menos aleatorias.

In [ ]:
response_top_k_1 = generator(prompt, max_length=50, top_k=1, num_return_sequences=5,pad_token_id=50256)
for r in response_top_k_1:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of art

In [ ]:
response_top_k_10 = generator(prompt, max_length=50,num_return_sequences=3, top_k=10, pad_token_id=50256)
for r in response_top_k_10:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is unclear, but it's already taking shape.

"We've got a lot more to learn about AI and artificial intelligence than the last few years have," said Dr. Peter J. Lutz, an
---
The future of artificial intelligence in healthcare is uncertain, but at the time of writing, the number of companies and institutions investing in the development of machine learning and artificial intelligence (AGI) technology is still in its infancy, and there is still a significant
---
The future of artificial intelligence in healthcare is now uncertain, with advances in computer vision and robotics still far away and many of the problems in health technology still present. We can now see the potential for artificial intelligence at work, in our healthcare, and in
---


In [ ]:
response_top_k_50 = generator(prompt, max_length=50,num_return_sequences=3, top_k=50, pad_token_id=50256)
for r in response_top_k_50:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain but, with a huge amount of research coming out of big universities, we can confidently predict what technologies will be made available in the next couple of years.

At the current rate of development, there
---
The future of artificial intelligence in healthcare is in the scientific realm. However, with the continued development of a number of new, innovative AI technologies, there are still those who consider AI the new frontier. That is why we are introducing the New Directions®
---
The future of artificial intelligence in healthcare is bright. But technology is often very new and evolving, and the future is murky.

An early guide to future healthcare

Many aspects of the future of healthcare may be unknown until very late into the
---


**top_p** permite elegir entre palabras de mayor probabilidad hasta que la probabilidad acumulada alcanza el porcentaje especificado. Esto permite flexibilidad sin restringir a un número fijo de palabras.

**top_p = 0.9**: El modelo elige palabras hasta que la probabilidad acumulada alcance el 90%. Esta configuración puede generar respuestas variadas y permite más flexibilidad sin reducir demasiado las opciones.

In [ ]:
response_top_p = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_p=0.9, pad_token_id=50256)
for r in response_top_p:
  print(r['generated_text'])
  print('---')

**top_p = 0.5**: El modelo solo considera palabras con una probabilidad acumulada del 50%. Esto hace que las respuestas sean más directas y menos creativas, similar a una temperatura baja.

In [ ]:
response_top_p = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_p=0.5, pad_token_id=50256)
for r in response_top_p:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain.

In the meantime, there are a number of companies that are trying to develop AI that can help them improve their healthcare.

A few of these companies are:

AeroMed
---
The future of artificial intelligence in healthcare is uncertain.

In the past few years, researchers have begun to explore the potential of artificial intelligence to improve healthcare delivery. The goal is to develop a machine that can diagnose, treat, and treat complex diseases
---
The future of artificial intelligence in healthcare is uncertain. The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain. The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in
---


**top_p = 0.2**: El modelo solo considera palabras con una probabilidad acumulada del 20%. Esto hace que las respuestas sean mucho menos creativas, similar a una temperatura muy baja.

In [ ]:
response_top_p = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_p=0.2, pad_token_id=50256)
for r in response_top_p:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---
The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future of artificial intelligence in healthcare is uncertain.

The future
---


En este ejemplo estamos combinando **top_k=50** y **top_p=0.9** para permitir al modelo elegir entre las palabras más probables, pero no restringiéndolo demasiado.

In [ ]:
response_top_k_p = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_k=50, top_p=0.9,pad_token_id=50256)
for r in response_top_k_p:
  print(r['generated_text'])
  print('---')

The future of artificial intelligence in healthcare is at stake, with the emergence of "supercomputers" that can perform tasks such as diagnosis and treatment, and will help doctors better understand the needs of patients.

In his paper, "The Future
---
The future of artificial intelligence in healthcare is at stake."

The question, as we know, is whether or not the artificial intelligence industry will continue to grow and grow.
---
The future of artificial intelligence in healthcare is uncertain. For example, AI will be used to perform more complex tasks, such as driving a car, or to create AI-powered robots that can help people find their way home.

"The goal
---


### Generador en español

In [ ]:
pipe = pipeline("text-generation", model="DeepESP/gpt2-spanish")

In [ ]:
pipe("Voy a vivir la vida",max_length=100, num_return_sequences=5, pad_token_id=50256)

[{'generated_text': 'Voy a vivir la vida que está esperando, no tiene esperanzas de que sea capaz de volver a vivir. \n\n—Todo el mundo piensa que podría haber perdido. \n\n—La cuestión es, —continuó explicando—, que tú eres capaz de vivir un sueño, si tienes suerte alguna, de que no pase el tiempo con la persona que lo está esperando para no olvidarlo y se presente cuando en realidad lo necesita. \n\n—No puedo. \n\n—¿Y qué vas a hacer, a morir'},
 {'generated_text': 'Voy a vivir la vida. \n\n—No he acabado. \n\n—Amanece y descanse. \n\nSe hizo un silencio entre los dos. \n\n—No, no me hace falta —dijo el joven—. El primer avión es el último. ¡Salgamos todos a recogerlo! \n\nEl avión había sido lanzado hacia la cumbre. El sol estaba ya muy alto y cuando ya iban a despegar, ya la niebla seguía cayendo sobre ellos. \n\n—Está bien,'},
 {'generated_text': 'Voy a vivir la vida que es lo opuesto de esta vida, a este mundo en el que todo es algo más que una vida. \n\nCuando le pregunté cómo s

In [ ]:
pipe = pipeline("text-generation", model="mrm8488/spanish-gpt2")

In [ ]:
pipe("Voy a vivir la vida",max_length=100, num_return_sequences=5, pad_token_id=50256)

[{'generated_text': 'Voy a vivir la vida y luego voy a morir feliz" y esta es la historia.En el cuento alemán, el personaje principal, el único, es el narrador de la historia, el narrador del cuento, con la historia.Y en el cuento chino, esto es otro personaje.Y como el cuento chino, tiene la historia en el mismo orden en que la tenía antes de la historia.Pero cuando hablamos del cuento chino, es importante recordar que el cuento en alemán lo que la gente quiere decir'},
 {'generated_text': 'Voy a vivir la vida.¡Quiero vivir la vida!¡Quiero vivir la vida!¡Quiero vivir la vida!¡No me hagas reír, que me muero!¡Y no me importa morir!¡Quiero vivir ya!¡Quiero vivir la vida!Así, así.Cuando vuelvas, me pones en su lugar.Que se sienta como en su casa, que se vea por primera vez.Se va con las luces apagadas y va hacia a ella.Que vea al marido, es algo'},
 {'generated_text': 'Voy a vivir la vida que siempre me imaginé... como era y si todo lo que mi madre me hizo fue mentir, y como lo hizo con m

## Resumir texto

In [6]:
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):
   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/555M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/324 [00:00<?, ?it/s]

This checkpoint seem corrupted. The tied weights mapping for this model specifies to tie decoder.bert.embeddings.word_embeddings.weight to decoder.cls.predictions.decoder.weight, but both are absent from the checkpoint, and we could not find another related tied weight for those keys
EncoderDecoderModel LOAD REPORT from: mrm8488/bert2bert_shared-spanish-finetuned-summarization
Key                                                                   | Status     | 
----------------------------------------------------------------------+------------+-
encoder.embeddings.position_ids                                       | UNEXPECTED | 
decoder.bert.embeddings.position_ids                                  | UNEXPECTED | 
decoder.bert.encoder.layer.{0...11}.attention.output.dense.weight     | MISSING    | 
decoder.bert.encoder.layer.{0...11}.output.LayerNorm.weight           | MISSING    | 
decoder.bert.encoder.layer.{0...11}.attention.self.value.bias         | MISSING    | 
decoder.bert.encod

In [7]:
text="""
El gol que Linda Caicedo le marcó a Alemania en la fase de grupos fue elegido por votación como el mejor del Mundial Femenino de Australia y Nueva Zelanda 2023.
"El impresionante gol de Linda Caicedo en la fase de grupos contra Alemania ha sido votado como el Gol Hyundai del Torneo de la Copa Mundial Femenina de la FIFA Australia y Nueva Zelanda 2023™", dice el comunicado de la entidad.
"Después de recoger el balón justo dentro del área, la sensación adolescente se abrió paso entre dos defensores alemanes con una croqueta perfecta. Luego, con un movimiento fluido, lanzó un disparo que se metió en la esquina superior derecha de la portería de Merle Frohm para darle a Colombia la ventaja", fue la descripción de Fifa.
"El ganador del Gol Hyundai del Torneo fue determinado por una encuesta de fanáticos, con el esfuerzo de Caicedo superando el sublime tiro libre de la estrella panameña Martha Cox contra Francia por sólo el 0,5 por ciento de los votos", agrega el comunicado.
"Realmente lo vi. Se me vinieron miles de recuerdos de todo lo que he pasado y es increible todo. Agradecerles a Colombia, a mis compañeras, que siento que lo luchamos hasta lo último y bueno, muy feliz", dijo entonces Linda.
El gol tuvo un significado especial para Caicedo, una imagen que le dio la vuelta al mundo por lo que marcó entonces: la primera derrota de Alemania en fase de grupos desde 1995. La dos veces campeona del mundo no superó la primera fase
"No tengo palabras para describir lo que acabamos de hacer. Pero bueno, mucha mesura. Hay que pensar en lo que se viene, que es Marruecos, y ya pensar en los otros partidos", declaró Linda en zona mixta.
"""

In [8]:
generate_summary(text)

'segundos segundos segundos camarera camarera continuar continuar continuar raya raya segundos segundos encanto segundos segundos Edad segundos segundos mandado mandado segundosgrargrargrar McC segundos Edad Edad segundos punto punto continuar continuar interac As As Asér raya raya raya enseñóffuérfuérfuérf segundos segundos As Sudáfrica mandado Eugeniogrargrar Bolívar Bolívar Bolívar vicepresidente vicepresidentetidostidos segundos segundosgrar'

In [11]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


model_name = "csebuetnlp/mT5_m2m_crossSum_enhanced"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Corrected: Use convert_tokens_to_ids instead of a non-existent private method
get_lang_id = lambda lang: tokenizer.convert_tokens_to_ids(
    model.config.task_specific_params["langid_map"][lang][1]
)

target_lang = "spanish"

input_ids = tokenizer(
    [WHITESPACE_HANDLER(text)], # Texto a resumir
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    decoder_start_token_id=get_lang_id(target_lang),
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4,
)[0]

supress_warning = tokenizer.decode(
    output_ids, # Assign to a variable to suppress the warning if it's not used
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

# Regenerate summary to correctly decode
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2713410588.py:4: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


Loading weights:   0%|          | 0/284 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


<extra_id_96> El gol que le marcó a Alemania en la fase de grupos fue el mejor del Mundial Femenino de Australia y Nueva Zelanda.


**Nota:** Ver otros modelos en: https://huggingface.co/models?pipeline_tag=summarization&language=es&sort=trending

## Traducir

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
t="""
Daniel’e Junggust is pouring cold beers for the regulars at Tailgators bar in the coastal town of Weeki Watchi, but her mind is on her home a couple of minutes down the road.

Her ground floor apartment was inundated with water - she fears everything inside is ruined.

“I tried to get as much out beforehand as I possibly could,” she says. She stayed with her mother further inland last night.

Police have set up a checkpoint along the road, and she hasn’t yet been back home.

Like many along this coast, Daniel’e is both counting the damage and her blessings, thankful that friends and family are safe.

“Sometimes the universe takes things away from you,” she muses, “and hopefully you get something back in return.”
"""

In [ ]:
translator("""
Daniel’e Junggust is pouring cold beers for the regulars at Tailgators bar in the coastal town of Weeki Watchi, but her mind is on her home a couple of minutes down the road.

Her ground floor apartment was inundated with water - she fears everything inside is ruined.

“I tried to get as much out beforehand as I possibly could,” she says. She stayed with her mother further inland last night.

Police have set up a checkpoint along the road, and she hasn’t yet been back home.

Like many along this coast, Daniel’e is both counting the damage and her blessings, thankful that friends and family are safe.

“Sometimes the universe takes things away from you,” she muses, “and hopefully you get something back in return.”
""")

[{'translation_text': "Daniel'e Junggust está echando cervezas frías para los clientes habituales en el bar Tailgators en la ciudad costera de Weeki Watchi, pero su mente está en su casa a un par de minutos por la carretera. Su apartamento en la planta baja estaba inundado de agua - teme que todo dentro esté arruinado. “Traté de sacar todo lo que pude de antemano”, dice. Se quedó con su madre en tierra adentro anoche. La policía ha establecido un puesto de control a lo largo de la carretera, y todavía no ha vuelto a casa. Como muchos a lo largo de esta costa, Daniel'e está contando el daño y sus bendiciones, agradecida de que amigos y familiares estén a salvo. “A veces el universo te quita las cosas”, reflexiona, “y espero que recuperes algo a cambio”."}]

In [ ]:
pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

In [ ]:
pipe(t,src_lang='eng_Latn',tgt_lang='spa_Latn', max_length=1000)

[{'translation_text': 'Daniel Junggust está derramando cervezas frías para los clientes regulares en el Tailgators bar en la ciudad costera de Weeki Watchi, pero su mente está en su casa un par de minutos más abajo de la carretera. Su apartamento de la planta baja estaba inundado de agua - teme que todo dentro esté arruinado. Traté de sacarle todo lo que pude antes, dice. Se quedó con su madre más al interior anoche. La policía ha establecido un puesto de control a lo largo del camino, y aún no ha vuelto a casa. Como muchos a lo largo de esta costa, Daniele está contando el daño y sus bendiciones, agradecida de que amigos y familia estén a salvo. A veces el universo te quita las cosas,  ella reflexiona,  y espero que recibas algo a cambio.  '}]

Si queremos cargar el modelo directamente

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
#model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

## Análisis de sentimientos

### Biblioteca pipiline para análisis de sentimiento multilengua

In [ ]:
sentiment_m = pipeline(task = 'sentiment-analysis',
                     model = 'nlptown/bert-base-multilingual-uncased-sentiment',
                     tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
print (sentiment_m.model.config)

BertConfig {
  "_name_or_path": "nlptown/bert-base-multilingual-uncased-sentiment",
  "_num_labels": 5,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "finetuning_task": "sentiment-analysis",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "1 star",
    "1": "2 stars",
    "2": "3 stars",
    "3": "4 stars",
    "4": "5 stars"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "1 star": 0,
    "2 stars": 1,
    "3 stars": 2,
    "4 stars": 3,
    "5 stars": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_to

In [ ]:
results = sentiment_m('Esto está mal')
results

[{'label': '1 star', 'score': 0.6360456347465515}]

In [ ]:
sentiment_m(text)

[{'label': '4 stars', 'score': 0.25946688652038574}]

### Modelos entrenados en español
Exiten algunos modelos que han sido entrenados con español, como:


*   pysentimiento/robertuito-sentiment-analysis




In [ ]:
sentiment_pipeline = pipeline('text-classification',
                              model="pysentimiento/robertuito-sentiment-analysis")

In [ ]:
results = sentiment_pipeline('Esto está mal')
results

[{'label': 'NEG', 'score': 0.9360191226005554}]

Este no admite textos muy grandes, pero podemos dividirlo por oraciones

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sent_text = nltk.sent_tokenize(text)

In [ ]:
sent_text

['\nEl gol que Linda Caicedo le marcó a Alemania en la fase de grupos fue elegido por votación como el mejor del Mundial Femenino de Australia y Nueva Zelanda 2023.',
 '"El impresionante gol de Linda Caicedo en la fase de grupos contra Alemania ha sido votado como el Gol Hyundai del Torneo de la Copa Mundial Femenina de la FIFA Australia y Nueva Zelanda 2023™", dice el comunicado de la entidad.',
 '"Después de recoger el balón justo dentro del área, la sensación adolescente se abrió paso entre dos defensores alemanes con una croqueta perfecta.',
 'Luego, con un movimiento fluido, lanzó un disparo que se metió en la esquina superior derecha de la portería de Merle Frohm para darle a Colombia la ventaja", fue la descripción de Fifa.',
 '"El ganador del Gol Hyundai del Torneo fue determinado por una encuesta de fanáticos, con el esfuerzo de Caicedo superando el sublime tiro libre de la estrella panameña Martha Cox contra Francia por sólo el 0,5 por ciento de los votos", agrega el comunica

In [ ]:
sent=[]
for s in sent_text:
  sent.append(sentiment_pipeline(s))

In [ ]:
sent

[[{'label': 'POS', 'score': 0.5557050704956055}],
 [{'label': 'NEU', 'score': 0.5725243091583252}],
 [{'label': 'POS', 'score': 0.8099199533462524}],
 [{'label': 'POS', 'score': 0.4944000244140625}],
 [{'label': 'NEU', 'score': 0.5015159845352173}],
 [{'label': 'NEU', 'score': 0.5796695351600647}],
 [{'label': 'POS', 'score': 0.8154635429382324}],
 [{'label': 'POS', 'score': 0.9507564902305603}],
 [{'label': 'POS', 'score': 0.8774600028991699}],
 [{'label': 'NEG', 'score': 0.5056822299957275}],
 [{'label': 'POS', 'score': 0.6953572630882263}],
 [{'label': 'NEU', 'score': 0.5165379643440247}]]

In [ ]:
labels=np.array([s[0]['label'] for s in sent])
score=np.array([s[0]['score'] for s in sent])

In [ ]:
import numpy as np

In [ ]:
len(labels[labels=='POS'])

7

In [ ]:
np.mean(score[labels=='POS'])

0.7427231924874442

In [ ]:
len(labels[labels=='NEU'])

4

In [ ]:
np.mean(score[labels=='NEU'])

0.542561948299408

In [ ]:
len(labels[labels=='NEG'])

1

In [ ]:
np.mean(score[labels=='NEG'])

0.5056822299957275

In [ ]:
sentiment_es = pipeline(task = 'sentiment-analysis',
                     model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
                     tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        {"use_fast": False}
    ))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print (sentiment_es.model.config)

BertConfig {
  "_name_or_path": "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.32.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [ ]:
results = sentiment_es('Esto está mal')
results

[{'label': 'LABEL_1', 'score': 0.840549886226654}]

In [ ]:
sentiment_es(text)

[{'label': 'LABEL_1', 'score': 0.8128289580345154}]

## Modelos para chat

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
input_text = "Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

In [ ]:
outputs = model.generate(input_ids,max_new_tokens=1000)

In [ ]:
print(tokenizer.decode(outputs[0]))

<pad> The Prime Minister of the United Kingdom is Theresa May. Theresa May is the wife of David Cameron. David Cameron is the Prime Minister of the United Kingdom. So, the final answer is Theresa May.</s>


In [ ]:
input_text = "Who is the president in Colombia? Explain the political situation in the country"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000)
print(tokenizer.decode(outputs[0]))

<pad> President of Colombia</s>


## Modelos con template para chat
Algunos modelos están más actualizados que otros. Además permiten un template de chat para el tokenizer.

Acá vemos un ejemplo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, gradio, warnings
from datasets import load_dataset
from trl import SFTTrainer

### meta-llama/Meta-Llama-3-8B-Instruct


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert in history!"},
    {"role": "user", "content": "Who is the current UK Prime Minister? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

I'd be happy to explain who the current UK Prime Minister is and how they came to power. Here's a step-by-step guide:

1. The United Kingdom is a parliamentary democracy, which means that the head of government is the Prime Minister, who is responsible for advising the monarch (currently Queen Elizabeth II) on matters of state.

2. The Prime Minister is the leader of the political party or coalition of parties that has the most seats in the House of Commons, which is the lower chamber of the UK Parliament.

3. The House of Commons is made up of 650 elected Members of Parliament (MPs), who are chosen by the public to represent their constituencies.

4. The Prime Minister is typically the leader of the party that wins the most seats in a general election, which is held every five years or whenever the government loses a vote of confidence.

5. In the 2019 general election, the Conservative Party, led by Boris Johnson, won a majority of 80 seats, allowing them to form a government.

6. Bo

In [ ]:
messages_2 = [
    {"role": "system", "content": "You are an spanish professor and always answer in Spanish"},
    {"role": "user", "content": "Who is the current Prime Minister of the UK? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages_2,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

¡Hola! Como profesor de español, me alegra explicarte quién es el actual Primer Ministro del Reino Unido. ¡Vamos a hacerlo paso a paso!

Paso 1: El Reino Unido es un país ubicado en Europa occidental, compuesto por Inglaterra, Escocia, Gales y Irlanda del Norte.

Paso 2: El sistema político del Reino Unido es una monarquía constitucional, lo que significa que el monarca (actualmente, Isabel II) es la jefa de Estado, pero no tiene poder ejecutivo. En su lugar, el Primer Ministro es el jefe de gobierno.

Paso 3: El Primer Ministro del Reino Unido es elegido por el partido político que tenga la mayoría en el Parlamento del Reino Unido, que es la cámara legislativa del país.

Paso 4: En las elecciones generales del Reino Unido, los ciudadanos eligen a los miembros del Parlamento (conocidos como diputados) que representan a sus respectivos distritos electorales.

Paso 5: El partido político que tenga la mayoría


In [ ]:
input_text = "Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000,pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<|begin_of_text|>Who is the UK Prime Minister? Explain step by step
The UK Prime Minister is the head of government in the United Kingdom. Here's a step-by-step guide on how they are chosen:
Step 1: The General Election
The UK holds a general election every five years, unless the government loses a vote of no confidence or the House of Commons is dissolved. During the election, voters elect members of parliament (MPs) to represent their constituencies.

Step 2: The Party with the Most Seats
The party with the most seats in the House of Commons forms the government. This is usually the party with the largest number of MPs.

Step 3: The Leader of the Winning Party
The leader of the party with the most seats becomes the Prime Minister. This is usually the leader of the party that won the most seats in the general election.

Step 4: The Queen's Approval
The Queen must approve the appointment of the Prime Minister. This is a formal process, and the Queen meets with the leader of the winning

In [ ]:
input_text = "You are an spanish professor and always answer in Spanish. Who is the current UK Prime Minister? Explain step by step"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=1000,pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

### HuggingFaceH4/zephyr-7b-bet

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

As of my training data (last updated in March 2021), the current UK Prime Minister is Boris Johnson. Here's a step-by-step explanation:

1. The UK is a constitutional monarchy, which means that the monarch (currently Queen Elizabeth II) is the head of state, but the day-to-day running of the government is carried out by the Prime Minister, who is the head of the government.

2. The Prime Minister is appointed by the monarch, on the advice of the leader of the political party or coalition that has a majority in the House of Commons (the lower house of the UK Parliament).

3. Boris Johnson is a member of the Conservative Party, which won a large majority in the December 2019 general election.

4. Following the election, Johnson was invited by the Queen to form a new government, and he was sworn in as Prime Minister on 24 July 2019.

5. Johnson's appointment was expected, as he had been the frontrunner to replace Theresa May, who resigned as Prime Minister in May 2019 after failing to get

### mistralai/Mistral-7B-Instruct-v0.3

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert in history!"},
    {"role": "user", "content": "Who is the current UK Prime Minister? Explain step by step"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))